In [3]:
import pandas as pd
import re

In [ ]:
df = pd.read_csv('..eval_data/job_title_des.csv')
df.head()

,Unnamed: 0,Job Title,Job Description
0,0,Flutter Developer,We are looking for hire experts flutter develo...
1,1,Django Developer,PYTHON/DJANGO (Developer/Lead) - Job Code(PDJ ...
2,2,Machine Learning,"Data Scientist (Contractor)\n\nBangalore, IN\n..."
3,3,iOS Developer,JOB DESCRIPTION:\n\nStrong framework outside o...
4,4,Full Stack Developer,job responsibility full stack engineer – react...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2277 entries, 0 to 2276
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       2277 non-null   int64 
 1   Job Title        2277 non-null   object
 2   Job Description  2277 non-null   object
dtypes: int64(1), object(2)
memory usage: 53.5+ KB


In [6]:
df.drop(columns=['Unnamed: 0'])

sum(df['Job Title'].isna()),sum(df["Job Description"].isna())


(0, 0)

In [7]:

df["Job Description"] = df["Job Description"].apply(lambda row: re.sub(r"\t", r"", row).strip())
df["Job Description"] = df["Job Description"].apply(lambda row: re.sub(r"(\n|\r)+", r"\n", row).strip())
df["Job Description"] = df["Job Description"].apply(lambda row: re.sub(r" +", r" ", row).strip())

In [8]:
df_sample = df.sample(500,random_state=3)
df_sample.to_csv('../eval_data/job_descriptions_500.csv')

## Creating Synthetic Resumes from the Job descriptions

In [1]:

from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain_core.prompts import ChatPromptTemplate
import pandas as pd
from typing import Literal, Optional, Tuple
from pydantic import BaseModel, Field

In [ ]:
llm = ChatOpenAI(
    model = "gpt-4o-mini",
    api_key= ''
)

In [195]:
df = pd.read_csv('../eval_data/job_descriptions_500.csv')
job_titles = list(df['Job Title'])
job_descriptions = list(df['Job Description'])
job_descriptions_list = []
for i in range(len(job_titles)):
    job_descriptions_list.append(job_titles[i] + "\n" + job_descriptions[i])


In [196]:
job_descriptions_list = job_descriptions_list[400:500]

In [197]:
len(job_descriptions_list)

100

In [198]:
class ResumeContainer(BaseModel):
    """Contains 2 resumes generated from job description"""
    perfect_match: str = Field(..., 
                        description="Resume which is perfect match to the given job description.")
    slightly_relevant: str = Field(..., 
                        description="Resume which is slightly relevant to the given job description.")

In [199]:
llm_generated_response = []
system_message = """You are an AI assistant that generates Resumes given specific job description.
                  Generate 2 resumes for each job description so that one of the resume is an perfect match, while other is only slightly relevant.
                  Use a combination of skills, different industry/project work experience, education, and certifications to produce resume data.
                  You may add some KPIs to make work experience realistic.
                  Do not include any note or explanation of how you generate the resumes.
                  Structure it like a real resume. Having appropriate headings and content.
                  Make up your own information for personal info. """

for jobs in job_descriptions_list:
    user_message ="""
    Here is the Job Description (Note that all required skills may not be present in resume and some nonrelevant details can be present.
     The length of a resume should only be between 200 and 500 words. 
    {jobs}""" 
    prompt = ChatPromptTemplate.from_messages(
    [
        ('system',system_message),
        ('human',user_message)
    ]
)
    structured_llm = llm.with_structured_output(ResumeContainer)
    response =  structured_llm.invoke(jobs) 
    #response = chain.invoke([user_message,system_message])
    llm_generated_response.append(response)
                                   


In [200]:
ground_truth_resumes = []
generated_resumes = []
for response in llm_generated_response:
    ground_truth_resumes.append(response.perfect_match.replace("*",""))
    generated_resumes.append(response.perfect_match.replace("*",""))
    generated_resumes.append(response.slightly_relevant.replace("*",""))



In [201]:
len(job_descriptions_list),len(ground_truth_resumes)

(100, 100)

In [202]:
test_df = pd.DataFrame({'Job Description':job_descriptions_list,"Ground Truth":ground_truth_resumes})
test_df.to_csv('../eval_data/test_df_5.csv')
generated_df = pd.DataFrame({'Resumes':generated_resumes})
generated_df.to_csv('../eval_data/generated_df_5.csv')

In [16]:
gen1 = pd.read_csv('../eval_data/synthetic_dataset/generated_df_1.csv')
gen2 = pd.read_csv('../eval_data/synthetic_dataset/generated_df_2.csv')
gen3 = pd.read_csv('../eval_data/synthetic_dataset/generated_df_3.csv')
gen4 = pd.read_csv('../eval_data/synthetic_dataset/generated_df_4.csv')
gen5 = pd.read_csv('../eval_data/synthetic_dataset/generated_df_5.csv')



In [17]:
final_gen = pd.concat([gen1, gen2, gen3, gen4, gen5], axis=0).drop(columns=['Unnamed: 0'])

In [19]:
final_gen.to_csv('../eval_data/main_data/combined_generated_df.csv')